# Artificial Neural Network

### Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [4]:
tf.__version__

'2.1.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [6]:
x = dataset.iloc[:, 3:-1].values

In [9]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [10]:
y = dataset.iloc[:, -1].values

### Encoding categorical data

Label Encoding the "Gender" column

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [13]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [16]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [19]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [21]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [22]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [23]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [24]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [26]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Train on 8000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 72us/sample - loss: 0.3357 - accuracy: 0.8645
Epoch 2/100
8000/8000 [==============================] - 1s 75us/sample - loss: 0.3356 - accuracy: 0.8616
Epoch 3/100
8000/8000 [==============================] - 1s 70us/sample - loss: 0.3358 - accuracy: 0.8622
Epoch 4/100
8000/8000 [==============================] - 1s 68us/sample - loss: 0.3357 - accuracy: 0.8641
Epoch 5/100
8000/8000 [==============================] - 1s 68us/sample - loss: 0.3352 - accuracy: 0.8666
Epoch 6/100
8000/8000 [==============================] - 1s 68us/sample - loss: 0.3352 - accuracy: 0.8643
Epoch 7/100
8000/8000 [==============================] - 1s 69us/sample - loss: 0.3351 - accuracy: 0.8650
Epoch 8/100
8000/8000 [==============================] - 1s 68us/sample - loss: 0.3354 - accuracy: 0.8639
Epoch 9/100
8000/8000 [==============================] - 1s 69us/sample - loss: 0.3353 - accuracy: 0.8652
Epoch 10/100
8000/8000 [

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [37]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) 

[[False]]


### Predicting the Test set results

In [38]:
y_pred = ann.predict(x_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1507   88]
 [ 196  209]]


0.858